#Uso Basico

Para usar TensorFlow se necesita entender primero que es TensorFlow:

* Representa calculos como grafos.
* Ejecuta los grafos en el contexto de sesiones.
* Representa los datos como tensores.
* Mantiene los estados con variables.
* Usa feeds y fetches para obtener datos dentro y fuera de operaciones arbitrarias.

#Descripcion General

TensorFlow es un sistema de programacion el cual representa calculos como grafos. Los nodos en el grafo se llaman ops (forma resumida para decir operaciones). Un op toma cero o mas Tensores, realiza un calculo y produce cero o mas Tensores. Un Tensor es un tipo de arreglo multidimensional. Por ejemplo se puede representar un mini-batch de imagenes como un arreglo de 4 dimensiones de numeros de punto flotante con dimensiones [batch, height, width, channels].

Un grafo TensorFlow es una descripcion de calculos. Para realizar cualquier calculo el grafo debe ser inicilizado en una sesion. Una sesion coloca las ops del grafo en dispositivos tal como CPUs o GPUs y provee metodos para ejecutarlos. Estos metodos regresan Tensores producidos por ops como objetos ndarray de numpy en python y como tensorflow::Tensor instancias en C/C++.

#El Grafo de Calculos

Los programas de TensorFlow suelen estar estructurados en una fase de construccion la cual forma un grafo y una fase de ejecucion que utiliza una sesion para ejecutar las ops en el grafo.

Por ejemplo es comun crear un grafo para representar y entrenar una red neuronal en la fase de construccion y despues ejecutar varias veces un conjunto de ops de entrenamiento en el grafo durante la fase de ejecucion. 

#La construccion del Grafo

Para empezar a construir el grafo con ops que no necesitan ninguna entrada (ops fuente) como constantes y pasar las salidas de algun calculo a otro calculo.

Los constructores de los ops en la libreria de python regresan objetos que representan las salidas de las ops construidas. Se pueden pasar estos objetos a los constructores de otras ops como entradas.

La libreria de python TensorFlow provee un grafo por default con varios constructores de ops para agregar nodos el cual es suficiente para varias aplicaciones.

In [9]:
import tensorflow as tf

# Crea una op constante que produce una matriz de 1 X 2. 
# La op se agrega como un nodo al grafo por default. 
#
# El valor que regresa el constructor representa la salida de la op constante.
matrix1 = tf.constant([[3., 3.]])

# Crea otra constante que produce una matriz de 2 X 1.
matrix2 = tf.constant([[2.],[2.]])

# Crea una op Matmul que toma la matriz1 y la matriz2 como entradas.
# Se regresa un valor producto que representa el resultado de la 
# Multiplicacion de matrices.
product = tf.matmul(matrix1, matrix2)

El grafo por default ahora tiene tres nodos los cuales son dos ops constantes y un op matmul(). Para multiplicar en realidad las matrices y obtener el resultado de la multiplicacion se debe iniciar una sesion en el grafo.

#Iniciando una Sesion en el Grafo

Para iniciar un grafo se crea un objeto sesion si no se le mandan argumentos al constructor de la sesion se inicializa como un grafo por default.

In [10]:
# Inicia la sesion en el grafo por default.

sess = tf.Session()

# Para ejecutar la op matmul se llama a la funcion run() de la sesion, pasando como argumento product
# El cual representa la salida de la op matmul. Esto indica que se quiere obtener la salida de la op
# matmul.
#
# Todas las entradas necesitan una op que se ejecute automaticamente durante una sesion. Tipicamente se
# ejecuta en paralelo.
#
# La ejecucion run(product) implica tres ops en el grafo las cuales son dos constantes y un matmul.
#
# La salida de el op se regresa como un resultado tal como un objeto ndarray de numpy.

result = sess.run(product)

print(result)

# Se cierra la sesion cuando se ha terminado.

[[ 12.]]


Las sesiones deben de ser cerradas para liberar los recursos. Tambien se puede entrar a una sesion con un bloque la sesion se cerrara automaticamente al fin del bloque.

In [11]:
with tf.Session() as sess:
  result = sess.run([product])
  print(result)

[array([[ 12.]], dtype=float32)]


La implementacion de TensorFlow traduce la definicion del grafo en operaciones ejecutables distribuidas a traves de los recursos de la computadora tal como el CPU. 

#Uso Interactivo

Para el uso interactivo en entornos de Python tal como IPython se puede usar la clase InteractiveSession y las funciones Tensor.eval() y Operation.run(). Esto evita tener que mantener una variable durante la sesion.

In [18]:
# Se entra a una sesion interactiva de TensorFlow
import tensorflow as tf

sess = tf.InteractiveSession()

x = tf.Variable([1.0, 2.0])

a = tf.constant([3.0, 3.0])

# Se inicializa x usando la funcion run() para inicializar su op.
x.initializer.run()

# Se agrega una op para restar x de a. Se ejecuta y se imprime el resultado.
sub = tf.sub(x, a)
print(sub.eval())

# Se cierra la sesion cuando se haya terminado.

[-2. -1.]


#Tensores

Los programas TensorFlow usan un tensor como estructura de datos para representar todos los datos. Solo los tensores se pueden pasar entre ops en el grafo de calculos. Se puede pensar en un tensor de TensorFlow como un arreglo de n-dimensiones o una lista. Un tensor tiene un tipo estatico, un rango y un tamanio.

#Variables

Las variables mantienen su estado a travez de la ejecucion de ops en el grafo de calculos. El siguiente ejemplo muestra una variable usada como un contador.

In [16]:
# Se crea una variable que sera inicializada con el valor escalar de 0.
state = tf.Variable(0, name="counter")

# Se crea una op para agregarla a un estado.

one = tf.constant(1)
new_value = tf.add(state, one)
update = tf.assign(state, new_value)

# Las variables deben ser inicializadas ejecutando el init de la op despues de iniciar el grafo. 
# Primero se debe de agregar la init op al grafo.
init_op = tf.initialize_all_variables()

# Iniciar el grafo y ejecutar las ops
with tf.Session() as sess:
  # Ejecutar 'init' op
  sess.run(init_op)
  # Se imprime el valor inicial del estado
  print(sess.run(state))
  # Se ejecuta la op y actualiza el estado ademas de imprimirlo.
  for _ in range(3):
    sess.run(update)
    print(sess.run(state))

0
1
2
3


La op assign() en este codigo es una parte de la expresion en el grafo tal como la op add() entonces realmente no ejecuta la instruccion hasta que se run() ejecute la expresion.

Tipicamente la representacion de parametros en un modelo estadistico se representa como un conjunto de variables. Por ejemplo los pesos de una red neuronal se representarian como un tensor osea una variable. Durante el entrenamiento se actualiza este tensor ejecutando el entrenamiento en el grafo varias veces.

#Fetches

Para obtener los fetch (Recuperar) las salidas de las operaciones se ejecuta en el grafo run() se llama a un objeto de sesion y se pasa en la recuperacion del tensor. En el ejemplo anterior se obtuvo el fetch (recupero) un solo nodo (estado) pero tambien se pueden obtener los fetch de multiples tensores.

In [17]:
input1 = tf.constant(3.0)
input2 = tf.constant(2.0)
input3 = tf.constant(5.0)
intermed = tf.add(input2, input3)
mul = tf.mul(input1, intermed)

with tf.Session() as sess:
  result = sess.run([mul, intermed])
  print(result)

[21.0, 7.0]


Todas las ops necesitan producir los valores de las peticiones de los tensores estas se ejecutan una vez.

#Feeds

Los ejemplos vistos antes introducen los tensores como un calculo dentro del grafo asi guardando constantes y variables. TensorFlow tambien provee un mecanismo de feed para parchear un tensor directamente dentro de cualquier operacion en el grafo

Un feed reeplaza la salida temporalmente de una operacion con un valor de tensor. Se suplen datos feed como un argumento para la llamada de run(). El feed solo se usa cuando la llamada a run se pasa. En los casos de uso mas comunes se disenan operaciones que seran feed para crearlas se usan tf.placeholders().

In [19]:
input1 = tf.placeholder(tf.float32)
input2 = tf.placeholder(tf.float32)
output = tf.mul(input1, input2)

with tf.Session() as sess:
  print(sess.run([output], feed_dict={input1:[7.], input2:[2.]}))

[array([ 14.], dtype=float32)]


Exception AssertionError: AssertionError() in <bound method InteractiveSession.__del__ of <tensorflow.python.client.session.InteractiveSession object at 0x7f9d5007acd0>> ignored


Una operacion placeholder() genera un error si no se da soporte de feed.